<div style="display: flex; background-color: RGB(100,149,237);" >
<h1 style="margin: auto; padding: 30px; ">Mission 1 </h1>
</div>

<div style="display: flex; background-color: RGB(100,149,237);" >
<h3 style="margin: auto; padding: 30px; ">Imporation des librairies</h3>
</div>

In [33]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

<div style="display: flex; background-color: RGB(100,149,237);" >
<h3 style="margin: auto; padding: 30px; ">Imporation des données</h3>
</div>

In [34]:
customers = pd.read_csv("../data/olist_customers_dataset.csv")
geolocation = pd.read_csv("../data/olist_geolocation_dataset.csv")
items = pd.read_csv("../data/olist_order_items_dataset.csv")
payments = pd.read_csv("../data/olist_order_payments_dataset.csv")
reviews = pd.read_csv("../data/olist_order_reviews_dataset.csv")
orders = pd.read_csv("../data/olist_orders_dataset.csv")
products = pd.read_csv("../data/olist_products_dataset.csv")
sellers = pd.read_csv("../data/olist_sellers_dataset.csv")
translate = pd.read_csv("../data/product_category_name_translation.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../data/olist_customers_dataset.csv'

<div style="display: flex; background-color: RGB(100,149,237);" >
<h3 style="margin: auto; padding: 30px; "> Traduction des produits </h3>
</div>

In [ ]:
translate.head()

In [ ]:
translate_mapper = dict(zip(translate.product_category_name, translate.product_category_name_english))

In [ ]:
products['product_category_name'] = products.product_category_name.map(translate_mapper)

In [ ]:
products.product_category_name.head()

<div style="display: flex; background-color: RGB(100,149,237);" >
<h3 style="margin: auto; padding: 30px; ">Création de la Dataframe initiale</h3>
</div>

In [ ]:
df = pd.merge(customers, orders, on = "customer_id", how = "inner")
df = pd.merge(pd.merge(df, items, on = "order_id", how='inner'), products, on = "product_id", how="inner")
df.head()

<div style="display: flex; background-color: RGB(100,149,237);" >
<h3 style="margin: auto; padding: 30px; ">Recency</h3>
</div>

**Nous allons créer une colonne qui va nous permettre de savoir combien de jour s'est écouler entre la dernière commande d'un client par rapport à la dernière commande du dernier client.**

In [ ]:
date_columns = ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']

for col in date_columns:
    df[col] = pd.to_datetime(df[col], format='%Y-%m-%d %H:%M:%S')

In [ ]:
df.info()

In [ ]:
df.describe(include="all")

In [ ]:
df_max_purchase = df.groupby('customer_unique_id').order_purchase_timestamp.max().reset_index()
df_max_purchase.columns = ['customer_unique_id', 'MaxPurchaseDate']

df_max_purchase.head()

In [ ]:
df_max_purchase['Recency'] = (df_max_purchase['MaxPurchaseDate'].max() - df_max_purchase['MaxPurchaseDate']).dt.days

In [ ]:
df_max_purchase

In [ ]:
df_max_purchase['Recency'].describe()

Sur nos 95420 clients différents : 

    -En moyenne leurs dernières commandes était il y a 243 jour
    -Au plus tard il y a 728 jours
    -Et la médianne se trouve à 223 jours

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
sns.histplot(df_max_purchase['Recency'], bins=30)
plt.show()

<div style="display: flex; background-color: RGB(100,149,237);" >
<h3 style="margin: auto; padding: 30px; ">Frequence </h3>
</div>

**Nous calculons la fréquence de commande d'un client.**

In [ ]:
df_frequency = df.groupby('customer_unique_id').order_purchase_timestamp.count().reset_index()
df_frequency.columns = ['customer_unique_id','order_purchase_timestamp']

df_frequency.describe()

**il y a au total 95420 clients.</br>
la moyennes est de 1.18.</br>
écart-type est de 0.6214.</br>
la mediane nous montre que les clients ne commande qu'une seule fois.</br>
75 pourcent des clients ne commande que 1 seule fois.</br>
et au maximum il y a 24 commandes du meme client.</br>**

<div style="display: flex; background-color: RGB(100,149,237);" >
<h3 style="margin: auto; padding: 30px; ">Nombre de produit et type de produit</h3>
</div>

**Nous allons maintenant voir le nombre de produit et le type de produit que commande un client.**

In [ ]:
df

<div style="display: flex; background-color: RGB(100,149,237);" >
<h3 style="margin: auto; padding: 30px; ">Panier Moyen </h3>
</div>

**Panier moyen par client**

In [ ]:
df.order_id.value_counts()

In [ ]:
df.order_id.value_counts().mean()

In [ ]:
df.order_id.value_counts().describe()

In [ ]:
df_panier = df.groupby("customer_id")['price'].mean().reset_index()

df_panier

In [ ]:
df_panier.describe()

**il y a au total 98666 clients.</br>
la moyennes est de 125.91 Dollars </br>
écart-type est de 190.98.</br>
la mediane nous montre que les clients depense 79 Dollars.</br>
75 pourcent des clients depense 139 Dollars.</br>
et au maximum il un client a dépensé 6735 Dollars.</br>**

<div style="display: flex; background-color: RGB(100,149,237);" >
<h3 style="margin: auto; padding: 30px; ">Canal d'achat </h3>
</div>

In [40]:
df["nombre_vente"] = 0
df_category = df.groupby("product_category_name")['nombre_vente'].count().reset_index()

In [43]:
df_category

,product_category_name,nombre_vente
0,agro_industry_and_commerce,212
1,air_conditioning,297
2,art,209
3,arts_and_craftmanship,24
4,audio,364
...,...,...
66,stationery,2517
67,tablets_printing_image,83
68,telephony,4545
69,toys,4117


In [62]:
df_provisoire = df[["product_photos_qty","product_category_name"]]

In [63]:
df_category = pd.merge(df_category, test, on="product_category_name", how="inner")

In [64]:
df_category.drop_duplicates(inplace=True)
df_category

,product_category_name,nombre_vente,product_photos_qty
0,agro_industry_and_commerce,212,2.0
22,agro_industry_and_commerce,212,1.0
23,agro_industry_and_commerce,212,3.0
33,agro_industry_and_commerce,212,6.0
36,agro_industry_and_commerce,212,8.0
...,...,...,...
106843,watches_gifts,5991,6.0
107303,watches_gifts,5991,7.0
108397,watches_gifts,5991,8.0
108682,watches_gifts,5991,9.0


In [65]:
df_category = df_category.groupby("product_category_name").median()

In [66]:
df_category.sort_values("nombre_vente")

,nombre_vente,product_photos_qty
product_category_name,,
security_and_services,2.0,2.5
fashion_childrens_clothes,8.0,3.5
la_cuisine,14.0,1.5
cds_dvds_musicals,14.0,2.0
arts_and_craftmanship,24.0,3.5
...,...,...
computers_accessories,7827.0,5.0
furniture_decor,8334.0,6.5
sports_leisure,8641.0,6.0
